# IMPORTAR LAS LIBRERIAS

In [1]:
import numpy as np
import pandas as pd
import cloudpickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

# CARGAR LOS DATOS

## Ruta del proyecto

In [2]:
ruta_proyecto = 'C:/Users/user/Desktop/Master Data Science/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/01_LEADSCORING'

## Nombre del fichero de datos

In [3]:
nombre_fichero_datos = 'Leads.csv'

## Cargar los datos

In [4]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df =  pd.read_csv(ruta_completa,index_col='id', sep=';')

## Seleccionar solo las variables finales

### Cargar la lista de variables finales

In [5]:
nombre_variables_finales = ruta_proyecto + '/05_Resultados/' + 'variables_finales.pickle'

pd.read_pickle(nombre_variables_finales).sort_index().index.to_list()

['ambito_Marketing Management',
 'ambito_Select',
 'descarga_lm_No',
 'ocupacion_Unemployed',
 'ocupacion_Working Professional',
 'origen_Lead Add Form',
 'paginas_vistas_visita_mms',
 'score_actividad_mms',
 'score_perfil_mms',
 'tiempo_en_site_total_mms',
 'ult_actividad_Chat Conversation',
 'ult_actividad_Converted to Lead',
 'ult_actividad_Page Visited on Website',
 'ult_actividad_SMS Sent',
 'visitas_total_mms']

### Apuntar (manualmente) la lista de variables finales sin extensiones

In [6]:
variables_finales = ['ambito',
                     'descarga_lm',
                     'ocupacion',
                     'origen',
                     'paginas_vistas_visita',
                     'score_actividad',
                     'score_perfil',
                     'tiempo_en_site_total',
                     'ult_actividad',
                     'visitas_total'
                    ]

### Crear la matriz de variables procesos (excel)

En mi caso, creé una matriz en Excel.

### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

# ESTRUCTURA DE LOS DATASETS

## Eliminar registros

### Por duplicados

In [7]:
df.drop_duplicates(inplace = True)

### Por EDA

In [8]:
df = df.loc[(df.no_llamar != 'OTROS') & (df.no_enviar_email != 'Yes') & (df.ult_actividad != 'Email Bounced')] 

### Para x

Quedarse solo con las de la lista.

In [9]:
x = df[variables_finales].copy()

In [10]:
x.shape

(6840, 10)

### Para y

Especificar la target.

In [11]:
target = 'compra'

Crear el y.

In [12]:
y = df[target].copy()

# CREAR EL PIPELINE

## Instanciar calidad de datos

### Crear la función

In [13]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6840 entries, 660737 to 579533
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ambito                 6231 non-null   object 
 1   descarga_lm            6840 non-null   object 
 2   ocupacion              5148 non-null   object 
 3   origen                 6840 non-null   object 
 4   paginas_vistas_visita  6730 non-null   float64
 5   score_actividad        3872 non-null   float64
 6   score_perfil           3872 non-null   float64
 7   tiempo_en_site_total   6840 non-null   int64  
 8   ult_actividad          6750 non-null   object 
 9   visitas_total          6730 non-null   float64
dtypes: float64(4), int64(1), object(5)
memory usage: 587.8+ KB


In [14]:
def calidad_datos(df):
    #tipos
    temp = df.astype({'visitas_total': 'Int64'})
    #imputar moda
    def imputar_moda(variable):
        return(variable.fillna(variable.mode()[0]))

    var_imputar_moda = ['ocupacion','ambito']

    temp[var_imputar_moda] = temp[var_imputar_moda].apply(imputar_moda)
    
    #imputar valor
    var_imputar_valor = ['origen',
                         'ult_actividad',
                         'descarga_lm']

    valor = 'DESCONOCIDO'

    temp[var_imputar_valor] = temp[var_imputar_valor].fillna(valor)

    #imputar mediana
    var_imputar_mediana = ['paginas_vistas_visita',
                           'score_actividad',
                           'score_perfil',
                           'tiempo_en_site_total',
                           'visitas_total']

    def imputar_mediana(variable):
        if pd.api.types.is_integer_dtype(variable):
            return(variable.fillna(int(variable.median())))
        else:
            return(variable.fillna(variable.median()))

    temp[var_imputar_mediana] = temp[var_imputar_mediana].apply(imputar_mediana)

    def agrupar_cat_raras(variable, criterio = 0.02):
        #Calcula las frecuencias
        frecuencias = variable.value_counts(normalize=True)
        #Identifica las que están por debajo del criterio
        temp = [cada for cada in frecuencias.loc[frecuencias < criterio].index.values]
        #Las recodifica en 'OTROS'
        temp2 = np.where(variable.isin(temp),'OTROS',variable)
        #Devuelve el resultado
        return(temp2)

    var_agrupar_cat_rars = ['ambito',
                           'descarga_lm',
                           'ocupacion',
                           'origen',
                           'ult_actividad']
    for variable in var_agrupar_cat_rars:
        temp[variable] = agrupar_cat_raras(temp[variable],criterio = 0.02)

    #Winsorización manual
    temp['visitas_total'] = temp['visitas_total'].clip(0, 50)
    
    return(temp)

In [15]:
calidad_datos(x)

,ambito,descarga_lm,ocupacion,origen,paginas_vistas_visita,score_actividad,score_perfil,tiempo_en_site_total,ult_actividad,visitas_total
id,,,,,,,,,,
660737,Select,No,Unemployed,API,0.0,15.0,15.0,0,Page Visited on Website,0
660728,Select,No,Unemployed,API,2.5,15.0,15.0,674,Email Opened,5
660727,Business Administration,Yes,Student,Landing Page Submission,2.0,14.0,20.0,1532,Email Opened,2
660719,Media and Advertising,No,Unemployed,Landing Page Submission,1.0,13.0,17.0,305,OTROS,1
660681,Select,No,Unemployed,Landing Page Submission,1.0,15.0,18.0,1428,Converted to Lead,2
...,...,...,...,...,...,...,...,...,...,...
579622,Select,No,Unemployed,API,2.6,14.0,17.0,1409,SMS Sent,13
579615,Business Administration,No,Unemployed,Landing Page Submission,2.5,14.0,20.0,210,SMS Sent,5
579546,Media and Advertising,Yes,Unemployed,Landing Page Submission,2.0,14.0,19.0,238,SMS Sent,2


In [16]:
calidad_datos(x).isna().sum()

ambito                   0
descarga_lm              0
ocupacion                0
origen                   0
paginas_vistas_visita    0
score_actividad          0
score_perfil             0
tiempo_en_site_total     0
ult_actividad            0
visitas_total            0
dtype: int64

### Convertirla en transformer

In [17]:
hacer_calidad_datos = FunctionTransformer(calidad_datos)

## Instanciar transformación de variables

In [18]:
var_ohe = ['ambito','descarga_lm','ocupacion','origen','ult_actividad']
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')


var_mms = ['paginas_vistas_visita','score_actividad','score_perfil','tiempo_en_site_total','visitas_total']
mms = MinMaxScaler()

## Crear el pipe del preprocesamiento

### Crear el column transformer

In [19]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (mms, var_mms),
    remainder='drop')

### Crear el pipeline del preprocesamiento

In [20]:
pipe_prepro = make_pipeline(hacer_calidad_datos, 
                            ct)

## Instanciar el modelo

### Instanciar el algoritmo

In [21]:
modelo = LogisticRegression(n_jobs = -1, 
                       solver = 'saga',
                       penalty = 'l1',
                       C = 1,
                       )

### Crear el pipe final de entrenamiento

In [22]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo)

### Guardar el pipe final de entrenamiento

In [23]:
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'

ruta_pipe_entrenamiento = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento

with open(ruta_pipe_entrenamiento, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento, file)

### Entrenar el pipe final de ejecución

In [24]:
pipe_ejecucion = pipe_entrenamiento.fit(x,y)

In [25]:
pipe_ejecucion

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function calidad_datos at 0x000001B79D891B20>)),
                                 ('columntransformer',
                                  ColumnTransformer(transformers=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['ambito',
                                                                    'descarga_lm',
                                                                    'ocupacion',
                                                                    'origen',
                                                                    'ult_actividad']),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler(),
                                                                   ['paginas_vistas_visita',
                                                                    'score_actividad',
                                                                    'score_perfil',
                                                                    'tiempo_en_site_total',
                                                                    'visitas_total'])]))])),
                ('logisticregression',
                 LogisticRegression(C=1, n_jobs=-1, penalty='l1',
                                    solver='saga'))])

# GUARDAR EL PIPE

## Nombre del pipe final de ejecución

In [26]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

## Guardar el pipe final de ejecución

In [27]:
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)